In [2]:
import tiktoken as tk
import torch


In [3]:
##importing txt file

In [4]:
with open("Moby_Dick.txt", "r",encoding="utf-8") as f:
    raw_text=f.read()



In [5]:
print(len(raw_text))

1238224


In [6]:
##initilize the BPE tokenizer for gpt2

In [7]:
tokenizer=tk.get_encoding("gpt2")

In [8]:
##generating token ids

In [9]:
token_ids=tokenizer.encode(raw_text)

In [10]:
print(len(token_ids))

331688


In [11]:
print(token_ids)

[464, 4935, 20336, 46566, 286, 16540, 88, 11740, 26, 1471, 11, 383, 44772, 198, 220, 220, 220, 220, 198, 1212, 47179, 318, 329, 262, 779, 286, 2687, 6609, 287, 262, 1578, 1829, 290, 198, 1712, 584, 3354, 286, 262, 995, 379, 645, 1575, 290, 351, 2048, 645, 8733, 198, 10919, 15485, 13, 921, 743, 4866, 340, 11, 1577, 340, 1497, 393, 302, 12, 1904, 340, 739, 262, 2846, 198, 1659, 262, 4935, 20336, 13789, 3017, 351, 428, 47179, 393, 2691, 198, 265, 7324, 13, 70, 19028, 13, 2398, 13, 1002, 345, 389, 407, 5140, 287, 262, 1578, 1829, 11, 198, 5832, 481, 423, 284, 2198, 262, 3657, 286, 262, 1499, 810, 345, 389, 5140, 198, 19052, 1262, 428, 46566, 13, 198, 198, 19160, 25, 16540, 88, 11740, 26, 1471, 11, 383, 44772, 198, 198, 13838, 25, 25028, 5616, 4244, 198, 198, 26362, 3128, 25, 2901, 352, 11, 5878, 685, 68, 10482, 1303, 1983, 486, 60, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 4042, 2904, 6153, 25, 3269, 678, 11, 32190, 198, 198, 32065, 25, 3594, 198, 198,

In [12]:
tk_ids=[340, 11, 1577, 340, 1497, 393, 302, 12, 1904, 340, 739, 262, 2846, 198, 1659, 262, 4935, 20336, 13789, 3017, 351, 428, 47179, 393, 2691, 198, 265, 7324, 13, 70, 19028, 13, 2398, 13, 1002, 345, 389, 407, 5140, 287, 262, 1578, 1829, 11, 198, 5832, 481, 423, 284, 2198, 262, 3657, 286, 262, 1499, 810, 345, 389, 5140, 198, 19052, 1262, 428, 46566, 13, 198, 198, 19160, 25, 16540, 88, 11740, 26, 1471, 11, 383, 44772, 198, 198, 13838, 25, 25028, 5616, 4244, 198, 198, 26362, 3128, 25, 2901, 352, 11, 5878, 685, 68, 10482, 1303, 1983, 486, 60, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 4042, 2904]

In [13]:
### Decode method to decode the token ids

In [14]:
tk_text=tokenizer.decode(tk_ids)

In [15]:
print(tk_text)

 it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Moby Dick; Or, The Whale

Author: Herman Melville

Release date: July 1, 2001 [eBook #2701]
                Most recently


In [16]:
## step2: Creating slind window for input and target pairs

In [17]:
#lets create a dataset &dataloader

In [18]:
from torch.utils.data import Dataset, DataLoader

In [19]:
class gptdataset(Dataset):
    def __init__(self,txt,tokenizer,max_len,stride):
        self.input_ids=[]
        self.target_ids=[]

        token_ids_v1=tokenizer.encode(txt)

        for i in range(0, len(token_ids_v1)-max_len,stride):
            input_chunk=token_ids_v1[i:i+max_len]
            target_chunk=token_ids_v1[i+1:i+max_len+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
            return len(self.input_ids)

    def __getitem__(self,idx):
            return self.input_ids[idx], self.target_ids[idx]

In [20]:
#lets create a data loader to load the data

In [21]:
def gpt_dataloader_v1(txt,batch_size=4,max_len=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
    data_tokenizer=tk.get_encoding("gpt2")
    dataset=gptdataset(txt,data_tokenizer,max_len,stride)
    dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)

    return dataloader

In [22]:
raw_text_1 = "Hello world! This is a test sentence for GPT style dataset building."
loader = gpt_dataloader_v1(raw_text_1, batch_size=1, max_len=4, stride=4, shuffle=False)

data_iter = iter(loader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[15496,   995,     0,   770]]), tensor([[995,   0, 770, 318]])]


In [23]:
##LEts create vecoter embedding layer
##embedding layer=token_embedding+postion_embedding

In [24]:
##Token enbedding layer

In [25]:
vobac_size=50257
tk_dimenison=256

In [26]:
import torch.nn as nn

In [27]:
token_embedding_layer=torch.nn.Embedding(vobac_size,tk_dimenison)

In [28]:
print(token_embedding_layer)

Embedding(50257, 256)


In [29]:
max_len=4
dataloder=gpt_dataloader_v1(raw_text,batch_size=8,max_len=max_len,stride=4,shuffle=False)
data_iter = iter(dataloder)
inputs,target= next(data_iter)

In [30]:
print(inputs)

tensor([[  464,  4935, 20336, 46566],
        [  286, 16540,    88, 11740],
        [   26,  1471,    11,   383],
        [44772,   198,   220,   220],
        [  220,   220,   198,  1212],
        [47179,   318,   329,   262],
        [  779,   286,  2687,  6609],
        [  287,   262,  1578,  1829]])


In [31]:
tk_emdedding_layer_1=token_embedding_layer(inputs)

In [32]:
print(tk_emdedding_layer_1.shape)

torch.Size([8, 4, 256])


In [33]:
print(tk_emdedding_layer_1)

tensor([[[ 1.5596,  1.4311, -0.4895,  ...,  0.2395,  1.0164, -1.4632],
         [-0.8309, -0.1930,  0.6640,  ..., -0.0614,  0.9356, -0.3635],
         [ 2.0100, -1.3733,  1.2970,  ..., -0.1645,  0.4183,  0.3888],
         [-0.6540, -0.8189,  0.1852,  ..., -0.1721, -1.2283, -2.1277]],

        [[-0.1072,  0.7699,  2.0443,  ..., -0.0854, -0.4407,  0.1447],
         [-0.8679, -1.8907, -0.5414,  ..., -0.2159, -0.4577, -0.3753],
         [ 0.2955, -0.8467, -0.4296,  ...,  0.8957, -1.3447,  0.2142],
         [-0.2726, -0.3625,  0.5228,  ..., -0.2958,  0.6850, -0.0374]],

        [[-0.5487, -0.0107, -0.5991,  ...,  2.3190,  0.4320, -0.6033],
         [ 1.9045,  0.5762, -2.1534,  ...,  0.7203, -0.0501,  0.7268],
         [-0.5882, -0.5428,  0.3933,  ..., -0.7051,  0.9897,  0.2720],
         [ 1.3742, -0.0624,  0.9357,  ...,  0.9975, -1.6848,  0.8039]],

        ...,

        [[ 0.4266, -1.0036,  0.2589,  ..., -0.2372,  1.2329, -1.1514],
         [-0.3249, -0.1443, -1.1126,  ..., -0.0779,  1.90

In [34]:
##LETS CREATE POSITIONAL EMBEDDING LAYER

In [35]:
context_len=max_len
pos_embeding=torch.nn.Embedding(context_len,tk_dimenison)
pos_embedding_layer=pos_embeding(torch.arange(context_len))

In [36]:
print(pos_embedding_layer.shape)

torch.Size([4, 256])


In [37]:
print(pos_embedding_layer)

tensor([[-1.1566,  0.1960, -2.2551,  ...,  0.8173,  1.0112, -0.7442],
        [-1.0004,  0.4385, -0.8315,  ..., -0.8623, -0.6163, -1.3550],
        [-0.0060, -0.8635, -0.2049,  ...,  1.0794, -0.2802,  1.9901],
        [-0.2212,  0.5660, -0.0865,  ...,  2.2350, -0.5586, -0.1718]],
       grad_fn=<EmbeddingBackward0>)


In [38]:
##Now lets create input embedding layer

In [39]:
input_embedding_layer=tk_emdedding_layer_1+pos_embedding_layer

In [40]:
print(input_embedding_layer.shape)

torch.Size([8, 4, 256])


In [42]:
print(input_embedding_layer)

tensor([[[ 0.4029,  1.6271, -2.7446,  ...,  1.0568,  2.0276, -2.2074],
         [-1.8312,  0.2455, -0.1675,  ..., -0.9237,  0.3192, -1.7186],
         [ 2.0040, -2.2367,  1.0921,  ...,  0.9149,  0.1381,  2.3789],
         [-0.8753, -0.2529,  0.0987,  ...,  2.0628, -1.7869, -2.2995]],

        [[-1.2638,  0.9659, -0.2109,  ...,  0.7319,  0.5705, -0.5995],
         [-1.8683, -1.4523, -1.3729,  ..., -1.0782, -1.0740, -1.7303],
         [ 0.2895, -1.7102, -0.6344,  ...,  1.9750, -1.6249,  2.2044],
         [-0.4939,  0.2035,  0.4363,  ...,  1.9392,  0.1264, -0.2092]],

        [[-1.7053,  0.1853, -2.8543,  ...,  3.1364,  1.4433, -1.3475],
         [ 0.9042,  1.0146, -2.9848,  ..., -0.1420, -0.6665, -0.6282],
         [-0.5943, -1.4063,  0.1885,  ...,  0.3743,  0.7095,  2.2622],
         [ 1.1530,  0.5037,  0.8492,  ...,  3.2324, -2.2435,  0.6321]],

        ...,

        [[-0.7301, -0.8076, -1.9962,  ...,  0.5801,  2.2441, -1.8956],
         [-1.3253,  0.2941, -1.9440,  ..., -0.9402,  1.28